# 📊 ניתוח נתונים - Exploratory Data Analysis (EDA)

## פרויקט גמר - ניתוח שוק ההון

---

**תוכן עניינים:**
1. תיאור הנתונים
2. טעינת הנתונים
3. סטטיסטיקות בסיסיות
4. ניתוח ערכים חסרים
5. התפלגות הנתונים
6. ניתוח קורלציות
7. ניתוח סדרות עתיות
8. אתגרים וממצאים

In [ ]:
# ייבוא ספריות
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# הגדרות תצוגה
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

COLORS = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3B1F2B']
print("✅ ספריות נטענו בהצלחה!")

---
# 1. תיאור הנתונים

## 1.1 מקור הנתונים

| מאפיין | תיאור |
|--------|--------|
| **מקור** | Yahoo Finance API (via yfinance library) |
| **סוג הנתונים** | נתוני שוק ההון - מחירי מניות ודוחות פיננסיים |
| **טווח זמן** | 5 שנים אחרונות (2019-2024) |
| **תדירות** | יומית (מחירים), רבעונית/שנתית (דוחות) |
| **היקף** | ~551 חברות (Top US companies) |
| **אחסון** | Google BigQuery |

## 1.2 טבלאות הנתונים

### טבלה 1: daily_prices - נתוני מחירים יומיים

| שדה | סוג | תיאור |
|-----|-----|--------|
| ticker | STRING | סימול המניה (AAPL, GOOGL) |
| date | DATE | תאריך המסחר |
| open | FLOAT64 | מחיר פתיחה |
| high | FLOAT64 | מחיר גבוה ביותר |
| low | FLOAT64 | מחיר נמוך ביותר |
| close | FLOAT64 | מחיר סגירה |
| volume | INT64 | נפח מסחר |
| dividends | FLOAT64 | דיבידנד |
| stock_splits | FLOAT64 | פיצול מניות |

### טבלה 2: income_statements - דוח רווח והפסד
### טבלה 3: balance_sheets - מאזן  
### טבלה 4: cash_flows - תזרים מזומנים

---
# 2. טעינת הנתונים

In [ ]:
# הגדרות BigQuery
PROJECT_ID = "pro-visitor-429015-f5"
DATASET_ID = "StockData"

client = bigquery.Client(project=PROJECT_ID)
print(f"✅ מחובר ל-BigQuery: {PROJECT_ID}")

In [ ]:
# בדיקת גודל הנתונים
query_sizes = f"""
SELECT 
    'daily_prices' as table_name,
    COUNT(*) as row_count,
    COUNT(DISTINCT ticker) as unique_tickers,
    MIN(date) as min_date,
    MAX(date) as max_date
FROM 
UNION ALL

SELECT 'income_statements', COUNT(*), COUNT(DISTINCT ticker), MIN(report_date), MAX(report_date)
FROM 
UNION ALL

SELECT 'balance_sheets', COUNT(*), COUNT(DISTINCT ticker), MIN(report_date), MAX(report_date)
FROM 
UNION ALL

SELECT 'cash_flows', COUNT(*), COUNT(DISTINCT ticker), MIN(report_date), MAX(report_date)
FROM """

df_sizes = client.query(query_sizes).to_dataframe()
print("📊 גודל הנתונים:")
display(df_sizes)
print(f"
סה"כ שורות: {df_sizes['row_count'].sum():,}")

In [ ]:
# טעינת נתוני מחירים
query_prices = f"""
SELECT * FROM ORDER BY ticker, date
"""

df_prices = client.query(query_prices).to_dataframe()
df_prices['date'] = pd.to_datetime(df_prices['date'])
print(f"✅ נטענו {len(df_prices):,} שורות")

---
# 3. סטטיסטיקות בסיסיות

In [ ]:
# סטטיסטיקות תיאוריות
print("📊 סטטיסטיקות - נתוני מחירים")
display(df_prices.describe())

In [ ]:
# חישוב תשואות יומיות
df_prices['daily_return'] = df_prices.groupby('ticker')['close'].pct_change()

# סטטיסטיקות תשואות לפי מניה
returns_stats = df_prices.groupby('ticker')['daily_return'].agg([
    ('mean_return', 'mean'),
    ('std_return', 'std'),
    ('min_return', 'min'),
    ('max_return', 'max'),
    ('count', 'count')
]).reset_index()

returns_stats['sharpe_ratio'] = (returns_stats['mean_return'] * 252) / (returns_stats['std_return'] * np.sqrt(252))
returns_stats['annual_volatility'] = returns_stats['std_return'] * np.sqrt(252) * 100

print("📈 Top 10 מניות לפי Sharpe Ratio:")
display(returns_stats.nlargest(10, 'sharpe_ratio'))

---
# 4. ניתוח ערכים חסרים

In [ ]:
# ניתוח ערכים חסרים
def analyze_missing(df, name):
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'column': missing.index,
        'missing_count': missing.values,
        'missing_pct': missing_pct.values
    })
    missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)
    
    print(f"
🔍 {name}: {len(missing_df)} עמודות עם חסרים")
    if len(missing_df) > 0:
        display(missing_df.head(10))
    return missing_df

missing_prices = analyze_missing(df_prices, 'daily_prices')

In [ ]:
# כיסוי נתונים לפי מניה
data_per_ticker = df_prices.groupby('ticker').agg({
    'date': ['min', 'max', 'count']
}).reset_index()
data_per_ticker.columns = ['ticker', 'start_date', 'end_date', 'trading_days']

print(f"📅 ממוצע ימי מסחר: {data_per_ticker['trading_days'].mean():.0f}")
print(f"מינימום: {data_per_ticker['trading_days'].min()}")
print(f"מקסימום: {data_per_ticker['trading_days'].max()}")

# מניות עם מעט נתונים
sparse = data_per_ticker[data_per_ticker['trading_days'] < 500]
print(f"
⚠️ מניות עם <500 ימים: {len(sparse)}")

In [ ]:
# התפלגות כמות ימי מסחר
fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(data_per_ticker['trading_days'], bins=50, color=COLORS[0], edgecolor='white')
ax.axvline(data_per_ticker['trading_days'].mean(), color='red', linestyle='--', 
           label=f"Mean: {data_per_ticker['trading_days'].mean():.0f}")
ax.axvline(500, color='orange', linestyle='--', label='Min threshold (500)')
ax.set_xlabel('מספר ימי מסחר')
ax.set_ylabel('מספר מניות')
ax.set_title('התפלגות כמות ימי המסחר')
ax.legend()
plt.show()

---
# 5. התפלגות הנתונים

In [ ]:
# התפלגות תשואות יומיות
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

returns = df_prices['daily_return'].dropna()

# היסטוגרמה
axes[0].hist(returns, bins=100, density=True, color=COLORS[0], alpha=0.7)
axes[0].axvline(returns.mean(), color='red', linestyle='--', label=f'Mean: {returns.mean():.4f}')

# Normal fit
x = np.linspace(-0.1, 0.1, 100)
axes[0].plot(x, stats.norm.pdf(x, returns.mean(), returns.std()), 'r-', lw=2, label='Normal')
axes[0].set_xlabel('תשואה יומית')
axes[0].set_title('התפלגות תשואות')
axes[0].legend()
axes[0].set_xlim(-0.15, 0.15)

# Q-Q Plot
stats.probplot(returns.sample(5000), dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot')
plt.tight_layout()
plt.show()

print(f"
Skewness: {returns.skew():.4f}")
print(f"Kurtosis: {returns.kurtosis():.4f}")
print("→ Kurtosis גבוה = זנבות שמנים (fat tails)")

---
# 6. ניתוח קורלציות

In [ ]:
# מטריצת קורלציה
sample_tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'NVDA', 'TSLA', 
                  'JPM', 'BAC', 'JNJ', 'XOM', 'WMT']

df_pivot = df_prices[df_prices['ticker'].isin(sample_tickers)].pivot(
    index='date', columns='ticker', values='close'
)
returns_pivot = df_pivot.pct_change().dropna()
corr_matrix = returns_pivot.corr()

# Heatmap
fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r',
            center=0, square=True, ax=ax)
ax.set_title('מטריצת קורלציה - תשואות יומיות')
plt.tight_layout()
plt.show()

In [ ]:
# זוגות עם קורלציה גבוהה/נמוכה
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append({
            'ticker1': corr_matrix.columns[i],
            'ticker2': corr_matrix.columns[j],
            'correlation': corr_matrix.iloc[i, j]
        })

corr_df = pd.DataFrame(corr_pairs).sort_values('correlation', ascending=False)
print("🔝 קורלציה גבוהה:")
display(corr_df.head(5))
print("
🔻 קורלציה נמוכה:")
display(corr_df.tail(5))

---
# 7. ניתוח סדרות עתיות

In [ ]:
# מחירים לאורך זמן
tickers_plot = ['AAPL', 'GOOGL', 'MSFT', 'NVDA', 'TSLA']

fig, axes = plt.subplots(len(tickers_plot), 1, figsize=(14, 3*len(tickers_plot)))

for idx, ticker in enumerate(tickers_plot):
    df_t = df_prices[df_prices['ticker'] == ticker].sort_values('date')
    ax = axes[idx]
    ax.plot(df_t['date'], df_t['close'], color=COLORS[idx % len(COLORS)])
    ax.set_ylabel('מחיר ($)')
    ax.set_title(f'{ticker}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# תנודתיות לאורך זמן
fig, ax = plt.subplots(figsize=(14, 5))

for ticker in ['AAPL', 'MSFT', 'TSLA']:
    df_t = df_prices[df_prices['ticker'] == ticker].sort_values('date')
    df_t['ret'] = df_t['close'].pct_change()
    vol = df_t.set_index('date')['ret'].rolling(30).std() * np.sqrt(252) * 100
    ax.plot(vol.index, vol, label=ticker, alpha=0.8)

ax.set_ylabel('תנודתיות שנתית (%)')
ax.set_title('תנודתיות מתגלגלת (30 יום)')
ax.legend()
plt.show()

---
# 8. סיכום אתגרים וממצאים

## 🔴 אתגרים:

1. **ערכים חסרים** בדוחות פיננסיים
2. **התפלגות לא-נורמלית** - זנבות שמנים
3. **כיסוי לא אחיד** - מניות עם מעט נתונים
4. **קורלציות משתנות** - עולות במשברים
5. **סקאלות שונות** - צריך נרמול

## 🟢 ממצאים:

1. קורלציות גבוהות בתוך סקטורים
2. עונתיות - הבדלים לפי ימים וחודשים
3. אשכולות תנודתיות (volatility clustering)
4. נפח גבוה מלווה בתנודתיות

## 📌 המלצות:

1. סינון מניות עם <500 ימי מסחר
2. Feature engineering - מדדים טכניים
3. מודלים: Decision Tree, XGBoost, HRP